<a href="https://colab.research.google.com/github/irinap9270/hg_ml/blob/main/Ch5-2.%20CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# """교차검증"""

In [3]:
# 데이터 불러옴
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 훈련 세트와 테스트 세트로 나눔
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# 검증 세트 생성
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
# 훈련 세트와 검증 세트 크기 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [4]:
# 훈련 세트로 모델 학습
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
# 훈련 세트와 검증 세트에 대한 정확도 출력
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [5]:
from sklearn.model_selection import cross_validate # 교차 검증을 위한 알고리즘
from sklearn.model_selection import StratifiedKFold
# 모델 객체, 훈련 세트 전체를 매개변수로 전달
# cv; 분할기(폴드를 어떻게 나눌지 결정)
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(scores)

# 5개의 교차 검증 점수를 평균한 최종 점수
import numpy as np
print(np.mean(scores['test_score']))

# 훈련 세트를 섞은 후 10-폴드 교차 검증 수행
# n_splits; 몇 폴드 교차 검증인지
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

{'fit_time': array([0.01263428, 0.0104866 , 0.01459169, 0.01162744, 0.01803803]), 'score_time': array([0.00182652, 0.00179243, 0.00177765, 0.0017705 , 0.00173211]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
0.8574181117533719


# """하이퍼파라미터 튜닝"""

In [6]:
from sklearn.model_selection import GridSearchCV # 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행하는 클래스
# 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 생성
# min_impurity_decrease; 노드를 분할하기 위한 불순도 감소 최소량
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# 그리드 서치 객체 생성
# 탐색 대상 모델과 params 변수를 전달
# n_jobs; 매개 변수에서 병렬 실행에 사용할 CPU 코어 수
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
# 학습이 끝나면 자동으로 검증 점수가 가장 높은 모델의 매개변수 조합으로 다시 학습

# 검증 점수가 가장 높은 모델 정확도 확인
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
# 최적의 매개변수 확인
print(gs.best_params_)
# 각 매개변수의 교차 검증 평균 점수 출력
print(gs.cv_results_['mean_test_score'])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [7]:
# 불순도 감소 최소량, 깊이 제한, 노드를 나누기 위한 최소 샘플 수 탐색
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [8]:
from scipy.stats import uniform, randint # 균등 분포에서 샘플링 하는 클래스(주어진 범위에서 고르게 값을 뽑음)
# uniform; 실수, randint; 정수

# 불순도 감소 최소량, 깊이 제한, 노드를 나누기 위한 최소 샘플 수, 리프 노드가 되기 위한 최소 샘플의 개수 탐색
# uniform과 randint를 이용
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

# n_iter; 샘플링 횟수
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
